In [1]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables

True

In [2]:
os.environ['GEMINI_API_KEY']=os.getenv("GEMINI_API_KEY")

In [15]:
from google import genai

client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

result = client.models.embed_content(
        model="embedding-001",
        contents="What is the meaning of life?")



In [16]:
result

EmbedContentResponse(embeddings=[ContentEmbedding(values=[-0.00026206698, -0.055920187, -0.012463197, -0.020672267, 0.0076786787, 0.0024069808, 0.030989334, 0.01582611, -0.015852105, 0.011664646, -0.009116539, 0.0010538554, 0.036472403, -0.03241015, 0.008458409, -0.0033494884, 0.021979261, 0.015039315, 0.011996514, -0.03020258, -0.03397108, 0.010469224, -0.0036774632, 0.01323687, 0.007085988, -0.03823258, 0.018632846, -0.07036457, -0.022376658, 0.05355737, -0.058401376, 0.035850685, -0.087846205, -0.01713318, 0.04601891, -0.06547601, 0.06361521, 0.003836632, -0.05621926, 0.0153549155, 0.027993219, -0.008997662, -0.07307445, -0.029412353, -0.007533671, 0.005410707, -0.008447231, 0.03168135, 0.0041648657, -0.10778426, 0.016246738, 0.034010187, 0.080236025, -0.007318589, -6.490005e-05, -0.007139853, 0.048286457, 0.015510907, -0.029625922, -0.0059952387, 0.008322641, 0.006501486, -0.025500132, 0.063725494, -0.019506635, -0.0068314015, 0.026189998, 0.022515899, 0.058747195, -0.018565234, 0.

In [17]:
# Extract the embedding vector
embedding = result.embeddings[0].values
embedding[:10] # print first 10

[-0.00026206698,
 -0.055920187,
 -0.012463197,
 -0.020672267,
 0.0076786787,
 0.0024069808,
 0.030989334,
 0.01582611,
 -0.015852105,
 0.011664646]

In [18]:
len(embedding)

768

In [ ]:
result = client.models.embed_content(
    model="embedding-001",
    contents=[
      'What is the meaning of life?',
      'How much wood would a woodchuck chuck?',
      'How does the brain work?'])

for embedding in result.embeddings:
    # Print just a part of the embedding to keep the output manageable
  print(str(embedding)[:50], '... TRIMMED]')
     

values=[-0.010632273, 0.019375853, 0.020965198, 0. ... TRIMMED]
values=[0.018468002, 0.0054281265, -0.017658807, 0 ... TRIMMED]
values=[0.058089074, 0.020941732, -0.10872878, -0. ... TRIMMED]
